### Regression Analysis

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

In [2]:
Products = ['KH 12 CT Original',
 'KH 12 CT Butter',
 'KH 4 CT Original',
 'KH Hamburger Buns',
 'KH Hot Dog Buns',
 'KH Sliced Bread',
 'KH Mini Subs',
 'KH Round Bread',
 'KH 12 CT Wheat',
 'KH 24 CT Original',
 'KH 4 CT Butter',
 'KH 18 CT',
 'KH Slider Buns',
 'KH 24 CT Butter']

In [3]:
resultsets = {}
for i in Products:
    print('\n')
    print(i)
    product = pd.read_csv(f'Tables/{i}.csv', index_col=0).reset_index(drop =True)
    y = product['Volume Sales']
    X = product.iloc[:,1:11]
    X = sm.add_constant(X)
    X = X.drop(columns=['Volume Sales Any Price Reduction','Volume Sales Any Feature','Volume Sales Any Display'])
    #X.loc[X['Volume Sales Any Price Reduction'] > 0, 'Volume Sales Any Price Reduction'] = 1
    #X.loc[X['Volume Sales Any Feature'] > 0, 'Volume Sales Any Feature'] = 1
    model = sm.OLS(y,X)
    result = model.fit()
    resultsets[i] = result
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV')
    print(result.summary())
    



KH 12 CT Original
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
                            OLS Regression Results                            
Dep. Variable:           Volume Sales   R-squared:                       0.771
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     5489.
Date:                Mon, 14 Dec 2020   Prob (F-statistic):               0.00
Time:                        18:01:02   Log-Likelihood:            -1.2461e+05
No. Observations:               11397   AIC:                         2.492e+05
Df Residuals:                   11389   BIC:                         2.493e+05
Df Model:                           7                                         
Covariance Type:            nonro

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
                            OLS Regression Results                            
Dep. Variable:           Volume Sales   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.911
Method:                 Least Squares   F-statistic:                 1.572e+04
Date:                Mon, 14 Dec 2020   Prob (F-statistic):               0.00
Time:                        18:01:02   Log-Likelihood:                -78034.
No. Observations:               10763   AIC:                         1.561e+05
Df Residuals:                   10755   BIC:                         1.561e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
                            OLS Regression Results                            
Dep. Variable:           Volume Sales   R-squared:                       0.692
Model:                            OLS   Adj. R-squared:                  0.692
Method:                 Least Squares   F-statistic:                     2759.
Date:                Mon, 14 Dec 2020   Prob (F-statistic):               0.00
Time:                        18:01:02   Log-Likelihood:                -91923.
No. Observations:                8613   AIC:                         1.839e+05
Df Residuals:                    8605   BIC:                         1.839e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                

### Prepare data for dashboard

In [4]:
displays = []
features = []
TPRs = []
features_TPRs = []
geo = []
CP_MD = []
SD = []
SB = []
SC = []
RC = []

for i in Products:
    product = pd.read_csv(f'Tables/{i}.csv', index_col=0).reset_index(drop =True)
    display = product.loc[product['Volume Sales Any Display'] > 0, 'Volume Sales Any Display'].mean()
    feature = product.loc[product['Volume Sales Any Feature'] > 0, 'Volume Sales Any Feature'].mean()
    TPR = product.loc[product['Volume Sales Any Price Reduction'] > 0, 'Volume Sales Any Price Reduction'].mean()
    feature_TPR = product.loc[(product['Volume Sales Any Price Reduction'] > 0)&(product['Volume Sales Any Feature'] >0), 'Volume Sales Any Price Reduction']\
            +product.loc[(product['Volume Sales Any Price Reduction'] > 0)&(product['Volume Sales Any Feature'] >0), 'Volume Sales Any Feature']
    displays.append(display)
    features.append(feature)
    TPRs.append(TPR)
    features_TPRs.append(feature_TPR.mean())
    geo.append(resultsets[i].params[2])
    CP_MD.append(resultsets[i].params[3])
    SD.append(resultsets[i].params[4])
    SB.append(resultsets[i].params[5])
    SC.append(resultsets[i].params[6])
    RC.append(resultsets[i].params[7])

In [5]:
byProduct = pd.DataFrame({'Product': Products, 'Display': displays, 'Feature': features, 'TPR': TPRs, 
                          'Feature&TPR': features_TPRs, 'Mass': geo, 'Chicken Program/Meal Deal': CP_MD, 
                          'Secondary Deli': SD, 'Secondary Bakery': SB, 'Secondary Center store': SC, 
                          'Retail Coverage': RC})
byProduct

,Product,Display,Feature,TPR,Feature&TPR,Mass,Chicken Program/Meal Deal,Secondary Deli,Secondary Bakery,Secondary Center store,Retail Coverage
0,KH 12 CT Original,5637.592444,11826.469819,5581.400933,24786.519978,78504.329022,1130.795670,-1182.585984,1653.565622,-367.467135,-137.745603
1,KH 12 CT Butter,1284.312209,2584.409326,1238.493303,5341.871835,25295.395712,268.051401,-406.056440,310.954137,-86.484523,128.079781
2,KH 4 CT Original,656.655324,1290.810543,362.213160,2930.008283,434.740724,34.474175,1.799252,-140.832394,319.444747,-81.298942
3,KH Hamburger Buns,438.822863,1530.115391,506.111189,3245.765936,1600.622605,238.402543,-144.564127,289.491336,45.943946,-28.907356
4,KH Hot Dog Buns,257.737845,861.566908,277.595379,1904.868037,359.103766,133.818742,-85.377704,118.596460,51.566404,15.705075
5,KH Sliced Bread,251.881494,365.056988,220.448545,889.170364,10475.461513,161.080334,-88.478723,82.170285,44.861679,55.464950
6,KH Mini Subs,586.487620,1103.661767,369.670352,2131.274004,19515.600855,213.963425,-134.615948,383.400999,1.699323,-139.412407
7,KH Round Bread,193.496544,765.347684,178.002368,1549.918022,4676.353379,85.365990,-83.689393,62.216036,-27.098383,-9.841366
8,KH 12 CT Wheat,357.332004,667.789155,444.856596,1340.746761,14179.560771,126.340712,-115.531865,73.515411,21.066227,70.917784
9,KH 24 CT Original,1825.100269,9101.540768,1213.468676,14822.656150,18835.137761,694.629595,-574.447443,763.496184,-465.818000,711.892614


In [6]:
byTactic = byProduct.T.iloc[1:,:].rename(columns=byProduct.T.iloc[0,:]).reset_index().rename(columns = {'index': 'Tactics'})

In [7]:
byTactic

,Tactics,KH 12 CT Original,KH 12 CT Butter,KH 4 CT Original,KH Hamburger Buns,KH Hot Dog Buns,KH Sliced Bread,KH Mini Subs,KH Round Bread,KH 12 CT Wheat,KH 24 CT Original,KH 4 CT Butter,KH 18 CT,KH Slider Buns,KH 24 CT Butter
0,Display,5637.59,1284.31,656.655,438.823,257.738,251.881,586.488,193.497,357.332,1825.1,239.102,621.292,255.494,729.642
1,Feature,11826.5,2584.41,1290.81,1530.12,861.567,365.057,1103.66,765.348,667.789,9101.54,587.937,2230.07,911.21,2147.98
2,TPR,5581.4,1238.49,362.213,506.111,277.595,220.449,369.67,178.002,444.857,1213.47,187.785,268.432,243.371,464.117
3,Feature&TPR,24786.5,5341.87,2930.01,3245.77,1904.87,889.17,2131.27,1549.92,1340.75,14822.7,1350.58,3217.6,1796.79,3454.94
4,Mass,78504.3,25295.4,434.741,1600.62,359.104,10475.5,19515.6,4676.35,14179.6,18835.1,-45.6983,49335.7,1918.58,584.326
5,Chicken Program/Meal Deal,1130.8,268.051,34.4742,238.403,133.819,161.08,213.963,85.366,126.341,694.63,34.5116,542.427,137.489,1002.42
6,Secondary Deli,-1182.59,-406.056,1.79925,-144.564,-85.3777,-88.4787,-134.616,-83.6894,-115.532,-574.447,-46.6816,-2128.68,-9.04402,-1031.64
7,Secondary Bakery,1653.57,310.954,-140.832,289.491,118.596,82.1703,383.401,62.216,73.5154,763.496,-38.2382,2929.66,173.136,1883.07
8,Secondary Center store,-367.467,-86.4845,319.445,45.9439,51.5664,44.8617,1.69932,-27.0984,21.0662,-465.818,62.3583,-1994.77,-46.0016,-1076.15
9,Retail Coverage,-137.746,128.08,-81.2989,-28.9074,15.7051,55.4649,-139.412,-9.84137,70.9178,711.893,35.3513,1089.75,-183.124,196.156


### Building Dashboard

In [8]:
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [10]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.Label('Graphs'),
        dcc.RadioItems(
            id = 'w_radio',
            options =[
                {'label': 'By Tactic', 'value':'0'},
                {'label': 'By Product', 'value': '1'}

            ],

            labelStyle={'display': 'inline-block'}
        ),  
        dcc.Dropdown(
            id = 'w_dropdown',
            options = [
                {'label':'Mass', 'value': 'Mass'},
                {'label':'Display', 'value': 'Display'},
                {'label':'Feature', 'value': 'Feature'},
                {'label':'TPR', 'value': 'TPR'},
                {'label':'Feature and TPR', 'value': 'Feature&TPR'},
                {'label':'Chicken Program/Meal Deal', 'value': 'Chicken Program/Meal Deal'},
                {'label':'Secondary Deli', 'value': 'Secondary Deli'},
                {'label':'Secondary Bakery', 'value': 'Secondary Bakery'},
                {'label':'Secondary Center store', 'value': 'Secondary Center store'},
                {'label':'Retail Coverage', 'value': 'Retail Coverage'}

            ],
            placeholder= 'Choose tactics...',
            searchable= True,
            multi=False,
            clearable=True,
            style={'width':'70%'}
        ),
        dcc.Dropdown(
            id = 'w_dropdown2',
            options = [
                {'label':'KH 12 CT Original', 'value': 'KH 12 CT Original'},
                {'label':'KH 12 CT Butter', 'value': 'KH 12 CT Butter'},
                {'label':'KH 4 CT Original', 'value': 'KH 4 CT Original'},
                {'label':'KH Hamburger Buns', 'value': 'KH Hamburger Buns'},
                {'label':'KH Hot Dog Buns', 'value': 'KH Hot Dog Buns'},
                {'label':'KH Sliced Bread', 'value': 'KH Sliced Bread'},
                {'label':'KH Mini Subs', 'value': 'KH Mini Subs'},
                {'label':'KH Round Bread', 'value': 'KH Round Bread'},
                {'label':'KH 12 CT Wheat', 'value': 'KH 12 CT Wheat'},
                {'label':'KH 24 CT Original', 'value': 'KH 24 CT Original'},
                {'label':'KH 4 CT Butter', 'value': 'KH 4 CT Butter'},
                {'label':'KH 18 CT', 'value': 'KH 18 CT'},
                {'label':'KH Slider Buns', 'value': 'KH Slider Buns'},
                {'label':'KH 24 CT Butter', 'value': 'KH 24 CT Butter'}


            ],
            placeholder= 'Choose product...',
            searchable= True,
            multi=False,
            clearable=True,
            style={'width':'70%'}
        ),
       
        html.Div([
            dcc.Graph(id = 'w_graph')
        ])
    ])
])

@app.callback(
        Output(component_id= 'w_graph', component_property='figure'),
        [Input(component_id= 'w_radio',component_property='value'),
         Input(component_id= 'w_dropdown', component_property='value'),
        Input(component_id= 'w_dropdown2', component_property='value')])


def generate_graph(w_radio, w_dropdown, w_dropdown2):
    if w_radio == '1':
        dff = byProduct
        barchart = px.bar(
                data_frame= dff,
                x='Product', y=w_dropdown,
                color=w_dropdown,
                
                title= f'The Impact of {w_dropdown} on Weekly In-store Volume Sale of Each Product ',

                labels={w_dropdown:'Volume Sale boosted'}, height=500)

    else:
        dff = byTactic
        barchart = px.bar(
                data_frame= dff,
                x='Tactics', y=w_dropdown2,
                color=w_dropdown2,
                title= f'The Impact of Each Tactic on Weekly In-store Volume Sale of {w_dropdown2} ',
                labels={w_dropdown2:'Volume Sale boosted'}, height=500)
        barchart.update_layout(
            showlegend = False
            )
    return barchart

if __name__ == '__main__':
    app.run_server(mode = 'inline', port = 8051)





